In [1]:
import pandas as pd
import numpy as np
from bokeh.plotting import figure, output_file, show
from bokeh.models import HoverTool, ColumnDataSource
from bokeh.io import output_notebook

In [2]:
output_notebook()

Loading BokehJS ...

In [19]:
dfAll = pd.read_csv('../data/processed/DataComparison.csv')
dfAll.head()

,COUNTY,DEQ_Cattle,DEQ_Hogs,DEQ_Poultry,EWG_Poultry,EWG_Hogs,EWG_Cattle,NASS_Poultry,NASS_Hogs,NASS_Cattle
0,ALAMANCE,2025.0,900.0,0.0,929430.0,600.0,1590.0,828163.0,166.0,15347.0
1,ALEXANDER,2950.0,0.0,0.0,3694536.0,NaN,650.0,6429966.0,91.0,18727.0
2,ALLEGHANY,2583.0,0.0,0.0,14239.0,NaN,2117.0,1150.0,159.0,18381.0
3,ANSON,1000.0,44614.0,0.0,4284486.0,46158.0,700.0,9464579.0,56359.0,9317.0
4,ASHE,NaN,NaN,NaN,28478.0,NaN,NaN,1513.0,NaN,17354.0


In [4]:
def crunch(df,xCol,yCol):
    #Subset the data and remove null values
    df = df.loc[:,[xCol,yCol]].dropna(how='any',axis='rows')
    x = df[xCol];y = df[yCol]

    #Compute the formula
    z = np.polyfit(x,y,1)
    #Create a formula
    px = np.poly1d(z)
    y_hat = px(x)
    
    #return x,y,and y_hat
    return(x,y,px)

In [17]:
def plot_it(df,src,commodity):
    #Generate data
    xCol = 'NASS_{}'.format(commodity)
    yCol = '{}_{}'.format(src,commodity)
    x,y,px = crunch(df,xCol,yCol)
    title = "{0} = {1} * {2:.3f} + {3:.0f}".format(yCol,xCol,px[1],px[0])
    #Source & hover
    source = ColumnDataSource(data = df)
    hover = HoverTool(tooltips = [("","@COUNTY")])
    #Plot
    p = figure(title=title,
               x_axis_label=xCol,
               y_axis_label=yCol,
               tools=[hover])
    p.scatter(x=xCol,y=yCol,size=8,source=df)
    p.line(x,px(x)) #Add trendline
    output_file('../scratch/{}_{}.html'.format(src,commodity))
    show(p)
    return

In [18]:
for src in ['EWG','DEQ']:
    for com in ['Poultry','Hogs','Cattle']:
        plot_it(dfAll,src,com)

In [5]:
#Generate data
xCol = 'NASS_Poultry'
yCol = 'EWG_Poultry'
x,y,px = crunch(dfAll,xCol,yCol)
title = "{0} = {1} * {2:.3f} + {3:.0f}".format(yCol,xCol,px[1],px[0])

#Source
source = ColumnDataSource(data = dfAll)
hover = HoverTool(tooltips = [("","@COUNTY")])
p = figure(title=title,
           x_axis_label=xCol,
           y_axis_label=yCol,
           tools=[hover])
p.scatter(x=xCol,y=yCol,size=8,source=dfAll)
p.line(x,px(x)) #Add trendline
show(p)

In [ ]:
#Create the figure
p = figure(title=str(px),
           x_axis_label=xCol,
           y_axis_label=yCol)
#p.scatter(x,y)
p.scatter(x=xCol,y=yCol,size=8,source=dfAll)
p.line(x,px(x)) #Add trendline
#output_file('../scratch/PoultryPlot.html')
show(p)

In [ ]:
p = figure(title='Fig1',x_axis_label='NASS',y_axis_label='DEQ')
p.scatter(x,y)
p.line(x,px(x))
show(p)